# Querying in SQL

## Setup

First of all, we need to suppress warnings, because there are a few frustrating but harmless warnings that come out of the `jupysql` package.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

Next, we install `jupysql` and load the `sql` jupyter extension.

In [2]:
%pip install -q jupysql
%load_ext sql

Finally, we need to connect to the database. The below command will connect to a SQLite database stored in the `data` directory.

In [11]:
%sql sqlite:///../data/chinook.sqlite

Now we are ready to start working with the database.

## Querying the database

To run SQL commands against the connected database, we start with `%sql` and then simply write our command.

In [12]:
%sql SELECT * FROM Artist;

Running query in 'sqlite:///../data/chinook.sqlite'

ArtistId,Name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains
6,Antônio Carlos Jobim
7,Apocalyptica
8,Audioslave
9,BackBeat
10,Billy Cobham


Sometimes our queries get quite large! To split the query over multiple lines, we type `%%sql` and then write the query.

In [13]:
%%sql --save top_artist
SELECT Artist.Name, COUNT(*) AS AlbumCount
FROM Album
JOIN Artist ON Album.ArtistId = Artist.ArtistId
GROUP BY Artist.Name
ORDER BY AlbumCount DESC
LIMIT 10;

Running query in 'sqlite:///../data/chinook.sqlite'

Name,AlbumCount
Iron Maiden,21
Led Zeppelin,14
Deep Purple,11
U2,10
Metallica,10
Ozzy Osbourne,6
Pearl Jam,5
Various Artists,4
Van Halen,4
Lost,4


Notice that we stored the query above, so we can reuse it

In [14]:
%sql SELECT * FROM top_artist LIMIT 3;

Generating CTE with stored snippets: 'top_artist'

Running query in 'sqlite:///../data/chinook.sqlite'

Name,AlbumCount
Iron Maiden,21
Led Zeppelin,14
Deep Purple,11


## Exploring tables

We can explore tables in HTML format by using `%sqlcmd explore`

In [15]:
%sqlcmd explore --table "Artist"

## Data profiling

To list all available tables, we have the `tables` command

In [16]:
%sqlcmd tables

Name
Album
Artist
Customer
Employee
Genre
Invoice
InvoiceLine
MediaType
Playlist
PlaylistTrack


To explore a table, we have the `columns` command

In [17]:
%sqlcmd columns --table Track

name,type,nullable,default,primary_key
TrackId,INTEGER,False,None,1
Name,NVARCHAR(200),False,None,0
AlbumId,INTEGER,True,None,0
MediaTypeId,INTEGER,False,None,0
GenreId,INTEGER,True,None,0
Composer,NVARCHAR(220),True,None,0
Milliseconds,INTEGER,False,None,0
Bytes,INTEGER,True,None,0
UnitPrice,"NUMERIC(10, 2)",False,None,0


We can do simple data profiling with the `profile` command

In [19]:
%sqlcmd profile --table Track

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
count,3503,3503,3503,3503,3503,2525,3503,3503,3503
unique,3503,3257,347,5,25,852,3080,3501,2
top,nan,Wrathchild,nan,nan,nan,Steve Harris,nan,nan,nan
freq,nan,5,nan,nan,nan,80,nan,nan,nan
mean,1752.0000,nan,140.9295,1.2084,5.7254,nan,393599.2121,33510207.0654,1.0508
min,1,nan,1,1,1,nan,1071,38747,0.99
max,3503,nan,347,5,25,nan,5286953,1059546140,1.99
